# Pipe Center Finder

The `PipeCenterFinder` determines the center position and cross-sectional geometry of a cylindrical pipe from laser-cross wall intersection points observed at different z-positions (stage positions along the pipe axis).

## Motivation

In volumetric PIV camera calibration, the camera needs to be calibrated to a coordinate system inside the pipe. This requires knowing the physical pipe center -- but without a calibrated camera, we cannot directly measure positions in world coordinates. The `PipeCenterFinder` breaks this chicken-and-egg problem through a bootstrapping approach:

1. Move the laser stage to several z-positions and image the wall intersections.
2. Fit the pipe center $z_0$ from chord lengths -- this is already in **stage coordinates (mm)**, no camera calibration needed.
3. Move the stage to $z_0$, then use the fitted **(u, v) center** in image space to drive the x/y stage axes until the laser intersection sits at the desired image position.
4. Read the physical x/y positions from the stage encoders -- the pipe center is now known in world coordinates.

The camera calibration is the *output* of the overall process, not an input. Each axis stays in its native coordinate space (z in mm from the stage, u/v in pixels from the camera), so no cross-domain calibration is required at this step.

## Key Idea

A laser cross projected into a pipe produces two lines that intersect the inner pipe wall. By imaging these intersections at several z-positions, the chord length varies parabolically with z -- longest at the pipe center, shrinking towards the edges. Fitting this parabola yields the pipe center and elliptical cross-section parameters, with full uncertainty propagation.

![](./images/pipe-wall-intersections.png)

The image above shows four camera frames at different z-positions. The red markers indicate detected wall intersection points -- these are the input to the `PipeCenterFinder`.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Input Data

The `PipeCenterFinder` expects an `(N, 3)` array of wall intersection points, where each row is `[u, v, z]`:

- **u, v** -- pixel coordinates of a wall intersection in the camera image
- **z** -- the corresponding stage position (in mm)

Points are provided in pairs: for each z-position, there are two points (left and right wall intersection of one laser line). The array is ordered as consecutive pairs `[left_1, right_1, left_2, right_2, ...]`.

In [ ]:
wall_points = np.array(
    [  # u,    v,     z
        [403, 711, 17.5],  # pair 1: left wall
        [579, 88, 17.5],  #         right wall
        [394, 749, 22.5],  # pair 2
        [592, 47, 22.5],
        [395, 751, 27.5],  # pair 3
        [591, 45, 27.5],
        [404, 712, 32.5],  # pair 4
        [579, 85, 32.5],
    ]
)

## Visualizing the Wall Intersections

Before fitting, it helps to inspect the raw data. Each pair of points at the same z-position defines a **chord** across the pipe interior.

In [ ]:
fig, ax = plt.subplots()

pairs = wall_points.reshape((-1, 2, 3))
for pair in pairs:
    ax.plot(*pair[:, :2].T, "o-", label=f"z = {pair[0, 2]:.1f} mm", alpha=0.7)

ax.legend()
ax.set_xlabel("u (pixel)")
ax.set_ylabel("v (pixel)")
ax.set_title("Chord endpoints in image space")
ax.set_aspect("equal")

# actual image size
ax.set_xlim(0, 1280)
ax.set_ylim(0, 800)

## The Parabolic Relationship

The chord length $L$ between two wall intersections relates to the z-position through the pipe's circular (or elliptical) cross-section. If the pipe has semi-axis $a$ (in image space) and semi-axis $b$ (along z), then:

$$L^2 = 4a^2 \left[1 - \frac{(z - z_0)^2}{b^2}\right]$$

This means $L^2$ is a **downward-opening parabola** in $z$, with its vertex at the pipe center $z_0$. The maximum chord length occurs at $z_0$ and equals $2a$.

In [ ]:
left_points = wall_points[0::2]
right_points = wall_points[1::2]
chord_lengths = np.linalg.norm(left_points[:, :2] - right_points[:, :2], axis=1)
z = left_points[:, 2]

fig, ax = plt.subplots()
ax.scatter(z, chord_lengths)
ax.set_xlabel("z (mm)")
ax.set_ylabel("chord length (px)")
ax.set_title("Chord length vs. z-position")

## Fitting with `PipeCenterFinder`

The `PipeCenterFinder` automates this analysis. It takes the wall intersection points and an optional `uncertainty` (standard deviation in pixels, applied uniformly to all coordinates). Calling `fit()` performs the parabolic fit and returns a `PipeGeometry` result.

The `plot()` method shows:
- **Left panel:** $L^2$ vs $z$ with the fitted parabola and the estimated pipe center $z_0$
- **Right panel:** the chord endpoints and midpoints in image space, with a fitted midpoint line and the estimated center position

In [ ]:
from laser_cross_calibration.geometry import PipeCenterFinder

finder = PipeCenterFinder(wall_points, uncertainty=1)
finder.fit()

finder.plot();

## Numerical Results

The `summary()` method provides a compact overview of all fitted parameters with their propagated uncertainties:

In [ ]:
print(finder.summary())

## Validation: Ellipse Cross-Section

The `plot_ellipse()` method visualizes the fitted elliptical cross-section in the u-z plane with uncertainty bands.

Since the pipe geometry is known a priori (nearly circular, smooth walls, approximately known center), the fit result can be checked against physical expectations. This serves as a diagnostic for the entire measurement chain -- not just the fit itself:

- **Fitted $b$ vs. known radius:** If the semi-axis $b$ deviates significantly from the known pipe radius, something is wrong in the z-axis (stage calibration, alignment).
- **Symmetry about $z_0$:** An asymmetric parabola can indicate a tilted pipe or misaligned stage axis.
- **Individual point deviations:** A single point falling off the parabola flags a problem at that specific z-position (reflection artifact, detection error, debris on the wall).

This makes the ellipse plot not just a visualization but a practical diagnostic tool -- problems in the physical or imaging setup surface as deviations from the expected geometry before a full calibration is attempted.

Below, the expected pipe diameter (30 mm) is overlaid as dashed lines at $z_0 \pm r$.

In [ ]:
EXPECTED_PIPE_DIAMETER = 30  # mm
expected_radius = EXPECTED_PIPE_DIAMETER / 2
z0 = finder.geometry.z0.nominal_value

ax = finder.plot_ellipse()
ax.axhline(z0 + expected_radius, ls="--", c="k", label="expected wall")
ax.axhline(z0 - expected_radius, ls="--", c="k")
ax.legend()